## PART 2 - DATA CLEANING
Part 2 of 2 for preprocessing data for DineSafe Gentrification project.

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import time

# Set display options
pd.set_option("display.max_rows", 500)

# Load pre-processed Dinesafe Dataframe:
with open('Data/preprocessed_ds.csv','r') as f:
    ds = pd.read_csv(f, index_col = None)
    
ds[:20]

,Year,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,ESTABLISHMENT_ID,ESTABLISHMENT_NAME,INSPECTION_ID,LATITUDE,LONGITUDE,risk_level
0,2012,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,102778249,43.832617,-79.266886,2
1,2013,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,103071804,43.832617,-79.266886,2
2,2014,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,103385541,43.832617,-79.266886,2
3,2015,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,103460005,43.832617,-79.266886,2
4,2016,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,103873791,43.832617,-79.266886,2
5,2017,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,103985362,43.832617,-79.266886,2
6,2018,Restaurant,5641 STEELES AVE E,10185207,TIM HORTONS,104152523,43.832617,-79.266886,2
7,2012,Institutional Food Service,15 BARBERRY PL,10185216,AMICA AT BAYVIEW,102877974,43.766412,-79.384040,3
8,2013,Retirement Homes(Licensed),15 BARBERRY PL,10185216,AMICA AT BAYVIEW,102975081,43.766412,-79.384040,3
9,2014,Retirement Homes(Licensed),15 BARBERRY PL,10185216,AMICA AT BAYVIEW,103214685,43.766412,-79.384040,3



However, we have to better understand the data before making any assertations. While an establishment may appear to exist for all 5 years, it may have changed **Establishment_IDs** despite keeping the same name. Per conversations with a DineSafe inspector, this may be due to a change in ownership. (It may be worth noting that renovations have no impact on Establishment_ID.)

### Filtering Out Establishment Types
Our analysis aims to identify areas experiencing gentrification. Turnover from certain (more industrial) establishment types such as child care caterers and high school cafeterias would not be indicative of that so we will remove that. Below is a list of types we will remove:

In [2]:
types_to_remove = ["Child Care - Catered", "Child Care - Food Preparation","School Nourishment Program",
                   "Cafeteria", "Cafeteria - Public Access",
                   "Institutional Food Service", "Secondary School Food Services", "Elementary School Food services",
                   "Retirement Homes", "Nursing Home / Home for the Aged", # Retirement Homes(Un/Licensed)", 
                   "Hospitals & Health Facilities", "College/University Food services",
                   "Chartered Cruise Boats",
                   "Hot Dog Cart",
                   "Other Educational Facility Food Services",
                   "Fairs / Festivals / Special Occasions", # All Xmas market vendors
                   "Food Recovery Program", 
                   "Bed & Breakfast",
                   "Rest Home",
                   "Milk Pasteurization Plant", # Human Milk Donation Centre
                   "Food Court Vendor"]

print("Old {:,} vs new {:,}".format(len(ds), len(ds[~ds['ESTABLISHMENTTYPE'].str.contains("|".join(types_to_remove))])))
print("Total rows removed: {:,}".format(len(ds) - len(ds[~ds['ESTABLISHMENTTYPE'].str.contains("|".join(types_to_remove))])))

# Remove above types
ds = ds[~ds['ESTABLISHMENTTYPE'].str.contains("|".join(types_to_remove))]

Old 99,587 vs new 86,079
Total rows removed: 13,508


### Large Venues
Certain addresses are for large venues and stadiums (e.g. Exhibition Place / Rogers Centre) and have several franchise food vendors associated with them. These often have high turnover rates and have different characteristics than other 'stand-alone' addresses we're looking at. We'll remove these larger venues to focus the scope of our data to better visualize gentrification.

In [3]:
arenas = ["100 PRINCES BLVD","2 STRACHAN AVE", 
                    "1 BLUE JAYS WAY", # Rogers Centre
                   "40 CARL HALL RD", # Downsview Flea Market
                   "40 BAY ST", # ACC / Scotiabank Arena
                   "170 PRINCES BLVD"] # BMO Field
ds = ds[~ds['ESTABLISHMENT_ADDRESS'].str.contains("|".join(arenas))]

print("Rows remaining after removing Exhibition Place addresses: {:,}".format(len(ds)))

Rows remaining after removing Exhibition Place addresses: 83,520


## Building our Dataframe(s)

We'll need to format our data into dataframes to be able to geospatially show where turnover is highest. Our end goal is to build two dataframes which will show changes by year: 1.) by establishment id and 2.) by address. 


End goal: Build a dataframe by address to see changes by year (columns).

We will need two tables - one identifying when establishment_ids existed each year (binary). 

Table 1:
- Index = Establishment_Id
- Columns = id_2012, id_2013,... name_2012, name_2013,... change_2013, change_2014,... lat, lon

Table 2:
- Index = Establishment_Address
- Columns = total_turnovers_2013, total_turnovers_2014...


---

_Issues with the data:_
- _Missing gap year_
- _Establishment Id changes (despite same address)_
- _Establishment Id changes (despite same name)_

In [4]:
# Build a function that shows change/continuation in restaurants at each address.

def grab_history_id(id_):
    # Returns a singular row with details about each address in the following format:
    
    cols = ['exist_2012', 'exist_2013', 'exist_2014', 'exist_2015', 'exist_2016', 'exist_2017', 'exist_2018']
    
    mini_df = ds[ds['ESTABLISHMENT_ID'] == id_]
    list_ = []
    
    for x in range(2012,2019):
        if x in list(mini_df['Year']):
            list_.extend([1])
        else:
            list_.extend([0])
    
    # Fill in existence gaps. Note that this (entire function) is by establishment_id. If e_id is diff but name is same, will need to be fixed down the road in turnover
    # If there is more than 1 opening/closing, initiate filling in existence gaps.
    exist_list = list_

    change_cnt = 0
    for pos, num in enumerate(exist_list[1:]):
        if num != exist_list[pos]: # if number doesn't match previous number (would be 'pos-1' except exist_list started at position 1!)
            change_cnt += 1
    
    # Initiate filling in existence gaps
    if change_cnt > 1:
        # Find positions of odd numbers
        exist_pos = [pos for pos,i in enumerate(exist_list) if i == 1]
        
        new_exist = exist_pos
        for pos, i in enumerate(exist_pos[1:]):
            gap = (i - exist_pos[pos]) - 1
            
            to_add = 0
            for j in range(gap):
                to_add += 1
                new_exist.append(exist_pos[pos] + to_add)
        new_exist.sort()
    
        # Replace original Exist sequence with fixed sequence
        for i in new_exist:
            exist_list[i] = 1
        list_ = exist_list
    sub_df = pd.DataFrame([list_], columns = cols)
    
    # Attach ID, Name, Address, Hazard Level
    sub_df['id'] = id_
    sub_df['name'] = mini_df.loc[mini_df['ESTABLISHMENT_NAME'].first_valid_index(), 'ESTABLISHMENT_NAME']
    sub_df['address'] = mini_df.loc[mini_df['ESTABLISHMENT_ADDRESS'].first_valid_index(), 'ESTABLISHMENT_ADDRESS']
    sub_df['risk_level'] = mini_df.loc[mini_df['risk_level'].first_valid_index(), 'risk_level']
    return sub_df


def add_change(sub_df):
    # Flag for ID change year-over-year
    sub_df['change_2013'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2013'] == sub_df.loc[:,'exist_2012']]
    sub_df['change_2014'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2014'] == sub_df.loc[:,'exist_2013']]
    sub_df['change_2015'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2015'] == sub_df.loc[:,'exist_2014']]
    sub_df['change_2016'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2016'] == sub_df.loc[:,'exist_2015']]
    sub_df['change_2017'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2017'] == sub_df.loc[:,'exist_2016']]
    sub_df['change_2018'] = [0 if i else 1 for i in sub_df.loc[:,'exist_2018'] == sub_df.loc[:,'exist_2017']]

    return sub_df
    
print("Example of function output:")
grab_history_id(10579403)

Example of function output:


,exist_2012,exist_2013,exist_2014,exist_2015,exist_2016,exist_2017,exist_2018,id,name,address,risk_level
0,0,0,0,0,1,0,0,10579403,KENSINGTON VARIETY,56 KENSINGTON AVE,1


## Create Turnover Dataframes:
One dataframe for turnover by establishment ID. From that, group by address to create a "Turnover by Address" dataframe.

In [5]:
# Create Index of Establishment_IDs
id_index = list(set(ds['ESTABLISHMENT_ID']))
address_index = list(set(ds['ESTABLISHMENT_ADDRESS']))

turnover = pd.DataFrame()

start = time.time()
for id_ in id_index:
    turnover = turnover.append(grab_history_id(id_))

# Combine restaurants where name and address are the same but establishment_id are different.
turnover = turnover.groupby(by = ['name','address'], as_index=False)[['exist_2012','exist_2013','exist_2014',
                                                                      'exist_2015','exist_2016','exist_2017','exist_2018']].sum(axis=1)

# Replace >1 values with 1 to keep binary (true/false)
turnover = turnover.replace(2,1).replace(3,1).replace(4,1).replace(5,1)
turnover.reset_index(drop = True, inplace = True)

time_elapsed = time.time() - start

print("Time elapsed: {:.2f} minutes.".format(time_elapsed/60)) 

turnover = add_change(turnover)

turnover.to_csv('Data/turnover_id.csv', index = None)

turnover

Time elapsed: 3.36 minutes.


,name,address,exist_2012,exist_2013,exist_2014,exist_2015,exist_2016,exist_2017,exist_2018,change_2013,change_2014,change_2015,change_2016,change_2017,change_2018
0,"""FOOD APPEAL.CA INC.""",222 ISLINGTON AVE,0,0,0,0,1,0,0,0,0,0,1,1,0
1,(FAMOUS PLAYERS )CINEPLEX ENTERTAINMENT,2190 YONGE ST,0,0,1,0,0,0,0,0,1,1,0,0,0
2,0109 DESSERT + CHOCOLATE,2190 MCNICOLL AVE,0,0,1,1,0,0,0,0,1,0,1,0,0
3,1 PLUS 1 PIZZA,361 OAKWOOD AVE,0,0,1,1,1,1,1,0,1,0,0,0,0
4,1 PLUS 2 PIZZA AND WING,3260 DUNDAS ST W,1,1,1,1,1,0,0,0,0,0,0,1,0
5,1 PLUS 2 PIZZA AND WINGS,1540 DANFORTH AVE,1,1,1,1,0,0,0,0,0,0,1,0,0
6,1 PLUS 3 PIZZA AND WINGS,1798 JANE ST,1,1,1,1,1,0,0,0,0,0,0,1,0
7,100 HUMBER COLLEGE CAFE,100 HUMBER COLLEGE BLVD,1,1,1,1,0,0,0,0,0,0,1,0,0
8,100 KM FOODS INC.,4478 CHESSWOOD DR,1,0,0,0,0,0,0,1,0,0,0,0,0
9,100 POR SIENTO SALVADORENO,404 OLD WESTON RD,0,0,0,0,1,0,0,0,0,0,1,1,0


In [6]:
# Build Table_2:
turnover_address = turnover.groupby(['address'])[['exist_2012','exist_2013','exist_2014','exist_2015','exist_2016','exist_2017','exist_2018',
                                                  'change_2013','change_2014','change_2015','change_2016','change_2017','change_2018']].sum().reset_index()

# Join for lat/lon
turnover_address = turnover_address.join(turnover[['address']].set_index('address'), on = 'address').drop_duplicates('address')

# Add a "Total_turnover" column
turnover_address['total_turnover'] = turnover_address['change_2013'] + turnover_address['change_2014'] + turnover_address['change_2015'] + \
turnover_address['change_2016'] + turnover_address['change_2017'] + turnover_address['change_2018']

turnover_address.sort_values('total_turnover', ascending= False)

# Add a "Annual_turnover" column
change_years = 6
turnover_address['annual_turnover'] = turnover_address['total_turnover']/change_years

# Add "max_units" column (estimate)
# Ideally this would be the number of (restaurant) retail units instead, if we had a commercial real estate database.
turnover_address['max_units'] = turnover_address[['exist_2012','exist_2013','exist_2014',
                                                  'exist_2015','exist_2016','exist_2017','exist_2018']].max(axis=1)

# Add "change_index" -- weighted turnover (annual turnover / max_units)
# The higher the index, the higher the change rate.
# Note the methodology for calculating "change_201x": closure and opening each count as one. So if both happened in 
## one year, change that year would be '2'.
turnover_address['change_index'] = turnover_address['annual_turnover']/turnover_address['max_units']
turnover_address['change_index'].replace(np.inf, 0, inplace = True)

# Add "multi_units" binary (1 for more than 1 unit per address - assumes >2 is multi_unit)
turnover_address['multi_units'] = [1 if i > 2 else 0 for i in turnover_address['max_units']]

turnover_address

,address,exist_2012,exist_2013,exist_2014,exist_2015,exist_2016,exist_2017,exist_2018,change_2013,change_2014,change_2015,change_2016,change_2017,change_2018,total_turnover,annual_turnover,max_units,change_index,multi_units
0,1 ADELAIDE ST E,2,2,2,2,2,3,3,0,0,0,0,1,0,1,0.166667,3,0.055556,1
1,1 AUSTIN TER,2,2,1,1,1,2,2,0,1,0,0,1,0,2,0.333333,2,0.166667,0
2,1 AVONDALE AVE,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0.000000,2,0.000000,0
3,1 BALDWIN ST,0,1,1,1,1,1,1,1,0,2,0,0,0,3,0.500000,1,0.500000,0
4,1 BALMORAL AVE,1,1,1,1,1,1,1,0,0,2,0,0,0,2,0.333333,1,0.333333,0
5,1 BAXTER ST,0,1,0,1,2,1,1,1,1,1,1,1,0,5,0.833333,2,0.416667,0
6,1 BEDFORD RD,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0.000000,1,0.000000,0
7,1 BENVENUTO PL,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0.000000,1,0.000000,0
8,1 BESTOBELL RD,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0.166667,1,0.166667,0
9,1 BLOOR ST E,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0.166667,1,0.166667,0


In [7]:
pd.concat([turnover_address.iloc[:,0], turnover_address.iloc[:,14:18]], axis=1).sort_values('change_index', ascending =False)

,address,total_turnover,annual_turnover,max_units,change_index
3148,1808 EGLINTON AVE W,8,1.333333,1,1.333333
6969,3358 DUNDAS ST W,8,1.333333,1,1.333333
1063,1192 QUEEN ST E,8,1.333333,1,1.333333
11485,839 COLLEGE ST,7,1.166667,1,1.166667
8552,467 DANFORTH AVE,7,1.166667,1,1.166667
11047,760 ST CLAIR AVE W,7,1.166667,1,1.166667
7853,4 MANOR RD E,7,1.166667,1,1.166667
8180,425 EDDYSTONE AVE,7,1.166667,1,1.166667
231,1011 DUFFERIN ST,7,1.166667,1,1.166667
6335,3023 BATHURST ST,7,1.166667,1,1.166667


### Join 'Turnover by Address' dataframe to latlon_ref table for lat+lon coordinates

In [8]:
# Make quick reference table for latlon and address, using original ds table.
latlon_ref = ds.groupby('ESTABLISHMENT_ADDRESS')[['LATITUDE','LONGITUDE']].first().reset_index()

# Join tables
turnover_address = turnover_address.join(latlon_ref.set_index('ESTABLISHMENT_ADDRESS'), on='address', how = 'left', rsuffix = '_ref')

# Save to CSV.
turnover_address.to_csv('Data/turnover_address (no institutions or food court).csv', index= None)
pd.DataFrame(Counter(ds['ESTABLISHMENT_ADDRESS']).most_common(), columns = ["address","count"])#.to_csv('Data/counter_address.csv', index=None)

,address,count
0,300 BOROUGH DR,237
1,4700 KEELE ST,217
2,3401 DUFFERIN ST,199
3,100 KING ST W,170
4,25 THE WEST MALL,166
5,1571 SANDHURST CRCL,163
6,91 FRONT ST E,159
7,900 DUFFERIN ST,151
8,1800 SHEPPARD AVE E,141
9,555 REXDALE BLVD,133


### After Addresses have been Attributed to a Neighbourhood 
(Processed in QGIS)

In [9]:
with open('Data/turnover_address (QGIS_withneighbourhoods).csv', 'r') as f:
    turnover_nb = pd.read_csv(f, index_col = None)

turnover_nb
turnover_nb = turnover_nb.groupby('AREA_NAME')[['total_turnover','max_units']].sum().reset_index()
turnover_nb['annual_turnover'] = turnover_nb['total_turnover']/change_years
turnover_nb['change_index'] = turnover_nb['annual_turnover']/turnover_nb['max_units']
turnover_nb.to_csv('Data/turnover_nb_DRAFT.csv', index= None)

In [10]:
turnover_nb.sort_values(['change_index','annual_turnover'], ascending = False)

,AREA_NAME,total_turnover,max_units,annual_turnover,change_index
37,Eglinton East (138),170,89,28.333333,0.318352
108,Scarborough Village (139),156,82,26.000000,0.317073
78,Morningside (135),65,35,10.833333,0.309524
47,Guildwood (140),33,19,5.500000,0.289474
123,West Hill (136),173,103,28.833333,0.279935
132,Woburn (137),379,227,63.166667,0.278267
129,Willowdale East (51),342,205,57.000000,0.278049
85,Newtonbrook East (50),93,57,15.500000,0.271930
68,Lawrence Park North (105),138,85,23.000000,0.270588
39,Englemount-Lawrence (32),76,47,12.666667,0.269504


## Table for Addresses and Establishments
Create a reference table which shows all establishments associated with addresses.

In [11]:
address_est_ref = turnover.groupby('address')['name'].apply(list)
address_est_ref.to_csv('Data/establishment_by_address.csv')